# Getting Started with Transfer Learning

With Machine Learning, it is advantageous to build your own tuned model off the shoulders of a pre-trained model. This can save a lot of time and money, not to mention you may not even have enough data to build a great model from scratch. It's a bit appealing in my mind, because if you could plug in a state-of-the-art right infront of your starter-kit model for a huge performance boost, it almost seems silly not to. Perhaps I will learn of the trade-offs as I explore this subject further.

## Scenario 1: Fine Tuning the ConvNet

## Scenario 2: Feature extractor from ConvNet

In this scenario, you take the architecture of developed Convolutional Neural Network, and essentially chop of the last layers. You do this because those last layers do a lot of the final classification work, while what you want is the capabilities of the earlier layers of the pre-trained model. The earlier layers have more low-level representation of your input. For example with images, the earlier layers get excited over features such as edges, shapes and colors. You want to make use of these representations, then swap out the classification with your own 

In [5]:
# Pull in the PyTorch, Numpy, Data and Plotting Libraries

from __future__ import print_function, division

import sys
import imp
print(imp.find_module('matplotlib'))
print(sys.path)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

plt.ion() # interactive mode

(None, '/Users/joshua/anaconda/lib/python3.5/site-packages/matplotlib', ('', '', 5))
['', '/Users/joshua/anaconda/lib/python3.5/site-packages/xlwings-0.10.0-py3.5.egg', '/Users/joshua/anaconda/lib/python35.zip', '/Users/joshua/anaconda/lib/python3.5', '/Users/joshua/anaconda/lib/python3.5/plat-darwin', '/Users/joshua/anaconda/lib/python3.5/lib-dynload', '/Users/joshua/anaconda/lib/python3.5/site-packages/Sphinx-1.3.5-py3.5.egg', '/Users/joshua/anaconda/lib/python3.5/site-packages/setuptools-20.3-py3.5.egg', '/Users/joshua/anaconda/lib/python3.5/site-packages/torchvision-0.1.8-py3.5.egg', '/Users/joshua/anaconda/lib/python3.5/site-packages', '/Users/joshua/anaconda/lib/python3.5/site-packages/aeosa', '/Users/joshua/anaconda/lib/python3.5/site-packages/IPython/extensions', '/Users/joshua/.ipython']


AttributeError: module 'matplotlib' has no attribute 'cbook'